# FIR Filter Class Implementation

This is the final exercise for the course and will test all the knowledge that you have gathered during this semester. The final exercise consist of developing a series of functions/methods to implement a `FIR` class in Python. For this, three files were provided and are stored in the `Common` directory. A brief description of which function or method needs to be modified in each file is presented here:

```python
fir.py
"Main file where filter implementations are developed, consist of the `FIR` class that you will develop. For this file you need to work on all methods."
```

```python
fft.py
"Auxiliary file which consist of the `FFT` class that will help you to implement the `FIR` class from fir.py and zero_pad_fourier function from auxiliary.py. For this file you need to work on all functions."
```

```python
auxiliary.py
"Auxiliary file, which was created in a previous exercise, and you will be adding two new functions: 1)zero_pad_fourier and 2)zero_pad"
```

In order to succeed I recommend you to start with `fft.py` file, then with `auxiliary.py` and finish with the `fir.py` file.

In [ ]:
import sys
sys.path.insert(0, '../')

import numpy as np
import matplotlib.pyplot as plt
from Common import fir
from Common import fft
from Common import auxiliary
from Common import common_plots

In [ ]:
cplots = common_plots.Plot()
flt = fir.FIR()
fast_ft = fft.FFT()

## Part 1: Test the `common_plots.py` file
In this first part you will test how to use the `plot_frequency_response` method from the class `Plot` using a $sinc$ signal. 

Create a $sinc$ signal using `numpy` with normalized frequency of $f_c=0.20$, and a transition bandwidth of $0.04$. Be aware that when you implement your `shifted_sinc` method you can't use the `numpy` implementation of the `sinc` function, but for this test is correct to use it.

In [ ]:
fc = None #Write your code here
BW = None #Write your code here
M = None #Write your code here
i = np.arange(0,M,1)

h = None #Write your code here

H = np.fft.fft(h)
f = np.linspace(0,1,h.shape[0])

cplots.plot_frequency_response(np.absolute(H).reshape(-1,1), f) #Note that this is a two sided spectrum

## Part 2: Test your `fft.py` file
Now we will compare your `FFT` class against numpy, and see if both: one sided and two sided results match. First run this test to see if your `fft` function works correctly:

In [ ]:
h = np.sinc(2*i*fc)
N = h.shape[0]
L = 1024

fc = 0.20
BW = 0.04
M = int(4/BW)
i = np.arange(0,M,1)

h_zero_pad = np.append(h,np.zeros(L-N))

H_own_one_side = fast_ft.fft(h_zero_pad, one_sided=True)
H_own_two_side = fast_ft.fft(h_zero_pad, one_sided=False)
H_numpy = np.fft.fft(h_zero_pad)

assert np.allclose(H_own_one_side, H_numpy[0:L//2]), "Your implementation missmatches numpy's, check your code!"
print("Great work! One sided implementation rocks!")

assert np.allclose(H_own_two_side, H_numpy), "Your implementation missmatches numpy's, check your code!"
print("But is better to have a two sided implementation! Excellent work!")

Now let's run this code and see if the function `ifft` also works well:

In [ ]:
h_own_two_side = fast_ft.ifft(H_own_two_side).astype('complex')
h_own_two_side[0] = h_own_two_side[0]/2

plt.plot(np.real(h_own_two_side[0:N]))
plt.plot(h)

assert np.allclose(h_own_two_side[0:M], h), "It seems there is an error, check your ifft function"
print("You have been doing a great work! Your IFFT is doing an amazing job!")

## Part 3: Test your `auxiliary.py` file
At this moment you can test your `zero_pad_fourier` method and check if both methods `DFT` and `FFT` works. The provided code test against numpy's implementation of the Fast Fourier Transform:

In [ ]:
fc = 0.20
BW = 0.04
M = int(4/BW)
i = np.arange(0,M,1)
L = 2048

h = np.sinc(2*i*fc)
h_zero_pad = np.append(h,np.zeros(L-N))
H_numpy = np.fft.fft(h_zero_pad).reshape(-1,1)
f = np.linspace(0,0.5,L//2)

H_dft, f_dft = auxiliary.zero_pad_fourier(h.reshape(-1,1), M, method='DFT', L=L)
H_fft, f_fft = auxiliary.zero_pad_fourier(h.reshape(-1,1), M, method='FFT', L=L)
print(H_dft.shape)

cplots.plot_frequency_response(np.absolute(H_numpy[0:L//2]), f, label='Numpy implemetation')
cplots.plot_frequency_response(np.absolute(H_dft), f_dft, label='DFT implemetation')
cplots.plot_frequency_response(np.absolute(H_fft), f_fft, label='FFT implemetation')

assert np.allclose(np.absolute(H_numpy[0:L//2]), np.absolute(H_fft)), "Your implementation missmatches numpy's, check your fft implementation."
print("Perfect! Both filters are doing an amazing job!")

## Part 4: Test your `fir.py` file
This is the last part of the Jupyter Notebook. In here you will test your window filter, then your low, high, pass band, and reject band filters.

### 4.1 Test your window filter
The first test we will perform is going to be on our window filters. We will check both `blackman` and `hamming` types and compare it's results for the spectral inversion and spectral reversal calculations.

In [ ]:
fc = 0.20
M = 103 #Now we use an odd number
L = 1024

hamming = flt.window_filter(fc, M, window_type='hamming', normalized=True)
blackman = flt.window_filter(fc, M, window_type='blackman', normalized=True)

# Spectral inversion 
hamming_inverted = flt.spectral_inversion(hamming)
blackman_inverted = flt.spectral_inversion(blackman)

# Spectral reversal
hamming_reversal = flt.spectral_reversal(hamming)
blackman_reversal = flt.spectral_reversal(blackman)

print(hamming.shape)

In [ ]:
# Fourier transformations for Hamming Filters
H, f_h = auxiliary.zero_pad_fourier(hamming, M)
H_inv, f_h_inv = auxiliary.zero_pad_fourier(hamming_inverted, M, method='FFT', L=L)
H_rev, f_h_rev = auxiliary.zero_pad_fourier(hamming_reversal, M, method='FFT', L=L)

# Fourier transformations for Blackman Filters
B, f_b = auxiliary.zero_pad_fourier(blackman, M)
B_inv, f_b_inv = auxiliary.zero_pad_fourier(blackman_inverted, M, method='FFT', L=L)
B_rev, f_b_rev = auxiliary.zero_pad_fourier(blackman_reversal, M, method='FFT', L=L)

In [ ]:
plt.rcParams["figure.figsize"] = (15, 5)

plt.subplot(1,3,1)
cplots.plot_frequency_response(np.absolute(H), f_h, label="Hamming")
cplots.plot_frequency_response(np.absolute(B), f_b, label="Blackman")
plt.legend()
plt.subplot(1,3,2)
cplots.plot_frequency_response(np.absolute(H_inv), f_h_inv, label="Inverted Hamming")
cplots.plot_frequency_response(np.absolute(B_inv), f_b_inv, label="Inverted Blackman")
plt.legend()
plt.subplot(1,3,3)
cplots.plot_frequency_response(np.absolute(H_rev), f_h_rev, label="Reversal Hamming")
cplots.plot_frequency_response(np.absolute(B_rev), f_b_rev, label="Reversal Blackman")
plt.legend()
plt.subplots_adjust(hspace=0.3)

### 4.2 Test your low pass filter
Now test your low pass filters. For this case we will use both `hamming` and `blackman` windows.

In [ ]:
fc = 0.2
M = 101
h_lp = flt.low_pass_filter(fc, M, window_type='hamming')
H_lp, fh_lp = auxiliary.zero_pad_fourier(h_lp, M)

b_lp = flt.low_pass_filter(fc, M, window_type='blackman')
B_lp, fb_lp = auxiliary.zero_pad_fourier(b_lp, M)

plt.rcParams["figure.figsize"] = (7, 5)
cplots.plot_frequency_response(np.absolute(H_lp), fh_lp, label='Hamming')
cplots.plot_frequency_response(np.absolute(B_lp), fb_lp, label='Blackman')
plt.legend()
plt.grid("on")

### 4.3 Test your high pass filter
It is time to test your high pass filter, besides testing again our `hamming` and `blackman` windows, we are also going to test the `spectral_inversion` and `spectral_reversal` methods.

In [ ]:
fc = 0.17
M = 200

h_hp = flt.high_pass_filter(fc, M, method='spectral_inversion', window_type='hamming')
H_hp, fh_hp = auxiliary.zero_pad_fourier(h_hp, M)

b_hp = flt.high_pass_filter(fc, M, method='spectral_reversal', window_type='blackman')
B_hp, fb_hp = auxiliary.zero_pad_fourier(b_hp, M)

cplots.plot_frequency_response(np.absolute(H_hp), fh_hp, label='Spectral Inversion-Hamming')
cplots.plot_frequency_response(np.absolute(B_hp), fb_hp, label='Spectral Reversal-Blackman')
plt.legend()
plt.grid("on")

### 4.4 Test your band/reject band filter
Finally we will create a band pass/reject filter and test our `band_filter` function like this:

In [ ]:
fc1 = 0.17
fc2 = 0.33

h = flt.band_filter(fc1, fc2, M, band_type='pass', window_type='blackman', method='spectral_reversal')
H, f = auxiliary.zero_pad_fourier(h, M)

g = flt.band_filter(fc1, fc2, M, band_type='reject', window_type='blackman', method='spectral_reversal')
G, f = auxiliary.zero_pad_fourier(g, M)

cplots.plot_frequency_response(np.absolute(H), f, label='Band Pass Filter')
cplots.plot_frequency_response(np.absolute(G), f, label='Band Reject Filter')
plt.legend()
plt.grid("on")